In [ ]:
import pandas as pd
import numpy as np

In [ ]:
EPOCHS = [10, 20]
LOGGING_INTERVAL = 500
MAX_GRAD_NORM = 0.1
LR = 1e-3

In [ ]:
BATCH_SIZES = [512]
BATCH_SIZE = BATCH_SIZES[0]
MAX_PHYSICAL_BATCH_SIZE = 8

In [ ]:
from transformers import RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification

def init_roberta(model_name="roberta-base", num_labels=2):
    """
    Initialize a RoBERTa model + tokenizer for sequence classification.
    """
    # Load config with custom number of labels
    config = RobertaConfig.from_pretrained(
        model_name,
        num_labels=num_labels,
    )

    # Load tokenizer
    tokenizer = RobertaTokenizer.from_pretrained(
        model_name,
        do_lower_case=False,
    )

    # Load model
    model = RobertaForSequenceClassification.from_pretrained(
        model_name,
        config=config,
    )

    return model, tokenizer

model, tokenizer = init_roberta()

In [ ]:
def make_trainable_layers(model, tokenizer):

  trainable_layers = [
      model.roberta.encoder.layer[-1],
      model.classifier
  ]

  total_params = 0
  trainable_params = 0

  for p in model.parameters():
      p.requires_grad = False
      total_params += p.numel()

  for layer in trainable_layers:
      for p in layer.parameters():
          p.requires_grad = True
          trainable_params += p.numel()

make_trainable_layers(model, tokenizer)


In [ ]:
import torch
import torch.nn as nn
import transformers
from torch.utils.data import TensorDataset
from transformers.data.processors.utils import InputExample
from transformers.data.processors.glue import glue_convert_examples_to_features

In [ ]:
def getDataSets(tokenizer):
  LABEL_LIST = [0,1]

  # ---- Compute a safe max length ----
  all_texts = pd.concat([train_file['text'], test_file['text'], poison_test_file['text']])
  # Use tokenizer to count ids (no special tokens here; we’ll add a tiny buffer)
  token_lengths = []
  for t in all_texts:
      t = "" if not isinstance(t, str) else t
      token_lengths.append(len(tokenizer.encode(t, add_special_tokens=True)))
  raw_max = max(token_lengths) + 2

  # Some tokenizers return a huge sentinel for model_max_length; clamp with 512 fallback
  model_max = getattr(tokenizer, "model_max_length", 512)
  if model_max is None or model_max > 100000:  # handle sentinel
      model_max = 512
  MAX_SEQ_LENGTH = min(raw_max, model_max)

  def _create_examples(df, set_type):
    examples = []
    for index, row in df.iterrows():
      if row["label"] not in LABEL_LIST:
        continue
      if not isinstance(row["text"], str):
        continue

      guid = f"{index}-{set_type}"
      examples.append(
          InputExample(guid=guid, text_a=row["text"], label=row["label"])
      )
    return examples

  def _df_to_features(df, set_type):
    examples = _create_examples(df, set_type)
    legacy_kwards = {}
    from packaging import version
    if version.parse(transformers.__version__) < version.parse("2.9.0"):
      legacy_kwards = {
          "pad_on_left": False,
          "pad_token": tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
          "pad_token_segment_id": 0,
      }
    return glue_convert_examples_to_features(
        examples = examples,
        tokenizer = tokenizer,
        label_list = LABEL_LIST,
        max_length = MAX_SEQ_LENGTH,
        output_mode = "classification",
        **legacy_kwards,
    )

  def _features_to_dataset(features):
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype = torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype = torch.long)

    all_labels = torch.tensor([f.label for f in features], dtype = torch.long)
    dataset = TensorDataset(all_input_ids, all_attention_mask, all_labels) # Removed all_token_type_ids
    return dataset

  train_features = _df_to_features(train_file, "train")
  test_features = _df_to_features(test_file, "test")
  poison_test_features = _df_to_features(poison_test_file, "test")

  train_dataset = _features_to_dataset(train_features)
  test_dataset = _features_to_dataset(test_features)
  poison_test_dataset = _features_to_dataset(poison_test_features)

  return train_dataset, test_dataset, poison_test_dataset